In [2]:
import pandas as pd
from pandas_profiling import ProfileReport


csv_file = "/home/maatrika/Desktop/NasaL2/L1Data/Merged_Kp.csv"  
df = pd.read_csv(csv_file)

profile = ProfileReport(df, title="Data Profiling Report", explorative=True)

report_file = "test_report.html"  
profile.to_file(report_file)

print(f"Data profiling report saved to {report_file}")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  3.09it/s]

Data profiling report saved to test_report.html


In [5]:
import pandas as pd

df = pd.read_csv('/home/maatrika/Desktop/NasaL2/L1Data/Merged_Kp.csv')


In [6]:
df.dropna(inplace=True)

In [7]:
df1 = df.copy()

In [8]:
df1.drop(columns=['Time'], inplace=True)

In [9]:
df1.columns

Index(['Bx', 'By', 'Bz', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
       'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak',
       'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au', 'av', 'aw',
       'ax', 'Kp'],
      dtype='object')

In [10]:
status_corr = corr_matrix = df1.corr()

In [11]:
corr_matrix['Kp']

a     0.083435
b    -0.042629
c    -0.074825
d     0.085428
e     0.088948
f    -0.031366
g    -0.032841
h     0.021722
i    -0.009589
j    -0.025887
k    -0.030413
l    -0.062703
m    -0.069344
n    -0.086171
o    -0.079996
p    -0.040275
q     0.035564
r     0.077692
s     0.152561
t     0.184131
u     0.186691
v     0.168159
w     0.177192
x     0.230340
y     0.251746
z     0.226487
aa    0.220054
ab    0.264909
ac    0.288093
ad    0.239016
ae    0.123457
af    0.136917
ag    0.115821
ah    0.111796
ai    0.078167
aj    0.046195
ak    0.024431
al    0.006126
am    0.022068
an   -0.012219
ao   -0.057389
ap   -0.042043
aq   -0.014172
ar    0.051358
as    0.026448
at   -0.043813
au   -0.043813
av   -0.043813
aw         NaN
ax         NaN
Kp    1.000000
Name: Kp, dtype: float64

In [12]:
def feature_selector_correlation(cmatrix, threshold):
    selected_features = []
    feature_score = []
    
    #kick the non-numeric columns
    numeric_columns = cmatrix.select_dtypes(include=['int64', 'float64'])
    
    for column_name in numeric_columns.columns:
        score = cmatrix[column_name].corr(cmatrix['Kp']) # correlation with Kp
        if abs(score) > threshold:
            selected_features.append(column_name)
            feature_score.append(['{:3f}'.format(score)])
    
    result = list(zip(selected_features, feature_score))
    return result

In [15]:
features_selected = feature_selector_correlation(status_corr, 0.5)
features_selected

[('w', ['0.533065']),
 ('x', ['0.564222']),
 ('y', ['0.560131']),
 ('z', ['0.532973']),
 ('aa', ['0.529855']),
 ('ab', ['0.542513']),
 ('ac', ['0.527724']),
 ('Kp', ['1.000000'])]

In [20]:
df1.isnull().sum()

Bx    0
By    0
Bz    0
a     0
b     0
c     0
d     0
e     0
f     0
g     0
h     0
i     0
j     0
k     0
l     0
m     0
n     0
o     0
p     0
q     0
r     0
s     0
t     0
u     0
v     0
w     0
x     0
y     0
z     0
aa    0
ab    0
ac    0
ad    0
ae    0
af    0
ag    0
ah    0
ai    0
aj    0
ak    0
al    0
am    0
an    0
ao    0
ap    0
aq    0
ar    0
as    0
at    0
au    0
av    0
aw    0
ax    0
Kp    0
dtype: int64

Feature selection using Random Forest Regressor

In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

df = df.replace('#NAME?', pd.NA)

numeric_columns = df.select_dtypes(include=['number'])

X = numeric_columns.drop(columns=['Kp']) 
y = df['Kp']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_imputed, y)

feature_importances = rf.feature_importances_

importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)


/home/maatrika/Desktop/NasaL2/nasa/lib/python3.10/site-packages/pandas/core/missing.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


   Feature    Importance
23       x  8.841962e-02
4        e  6.428053e-02
28      ac  5.722643e-02
20       u  5.019001e-02
19       t  4.916028e-02
17       r  4.716211e-02
24       y  4.495624e-02
18       s  3.959195e-02
3        d  3.525478e-02
0        a  3.317272e-02
10       k  3.176716e-02
27      ab  3.158427e-02
15       p  2.967889e-02
16       q  2.805320e-02
14       o  2.791863e-02
9        j  2.540232e-02
22       w  2.452991e-02
7        h  2.336991e-02
21       v  2.310765e-02
2        c  2.304090e-02
12       m  2.279546e-02
25       z  2.245268e-02
29      ad  2.162650e-02
13       n  2.150218e-02
8        i  2.026520e-02
11       l  1.983665e-02
26      aa  1.881938e-02
6        g  1.289605e-02
30      ae  9.706481e-03
5        f  9.694038e-03
31      af  8.685716e-03
32      ag  7.771004e-03
34      ai  5.826795e-03
35      aj  5.292988e-03
33      ah  4.958856e-03
37      al  3.251659e-03
36      ak  2.623759e-03
38      am  2.144707e-03
42      aq  6.053755e-04


Feature Selection using RFE Feature Ranking

In [28]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np

df1 = df.replace('#NAME?', np.nan)

X = df1.drop(columns=['Kp']) 
y = df1['Kp']

non_numeric_columns = X.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_columns)

X_numeric = X.select_dtypes(include=['number'])


imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rfe = RFE(estimator=rf, n_features_to_select=1)

rfe.fit(X_imputed, y)

feature_ranking = rfe.ranking_

ranking_df = pd.DataFrame({'Feature': X_numeric.columns, 'Ranking': feature_ranking})

ranking_df = ranking_df.sort_values(by='Ranking')

print(ranking_df)


/home/maatrika/Desktop/NasaL2/nasa/lib/python3.10/site-packages/pandas/core/missing.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


Non-numeric columns: Index(['Time', 'Bx', 'By', 'Bz'], dtype='object')
   Feature  Ranking
17       r        1
23       x        2
4        e        3
19       t        4
20       u        5
28      ac        6
10       k        7
24       y        8
3        d        9
18       s       10
15       p       11
27      ab       12
0        a       13
14       o       14
7        h       15
22       w       16
16       q       17
9        j       18
2        c       19
11       l       20
21       v       21
13       n       22
25       z       23
29      ad       24
12       m       25
26      aa       26
8        i       27
6        g       28
30      ae       29
31      af       30
5        f       31
32      ag       32
35      aj       33
34      ai       34
37      al       35
33      ah       36
38      am       37
36      ak       38
42      aq       39
1        b       40
39      an       41
40      ao       42
43      ar       43
41      ap       44
44      as       45
45      a

Feature Selection

### Method 1 - RF Feature Importance:

Features: 'x', 'e', 'ac', 'u', 't', 'r', 'y', 's', 'd', 'a', 'k'

### Method 2 - RFE Feature Ranking:

Features: "r", "x", "t", "u", "ac", "k", "y", "d", "s"

### Correlation Values with Kp:

1. 'x' - Correlation: 0.564222
2. 'y' - Correlation: 0.560131
3. 'ab' - Correlation: 0.542513
4. 'w' - Correlation: 0.533065
5. 'z' - Correlation: 0.532973
6. 'aa' - Correlation: 0.529855
7. 'ac' - Correlation: 0.527724


### Considering both the feature importance rankings and correlation values, you can prioritize the following features:

1. 'x' (Method 1 and Correlation)
2. 'y' (Method 2 and Correlation)
3. 'ac' (Method 1 and Correlation)
4. 'ab' (Correlation)
5. 'w' (Correlation)
6. 'z' (Correlation)
7. 'aa' (Correlation)
8. 'Bx', 'By', and 'Bz' 

GRU MODEL

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_csv('/home/maatrika/Desktop/NasaL2/L1Data/Merged_Kp.csv') 

numeric_columns = df.select_dtypes(include=[np.number])
X = numeric_columns.drop(columns=['Kp'])  
y = numeric_columns['Kp']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

#define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.gru(x.unsqueeze(2), h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out


input_size = X_train_tensor.shape[1]  
hidden_size = 128
num_layers = 2
output_size = 1

model = GRUModel(input_size, hidden_size, num_layers, output_size)

#define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#training loop
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    #backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    mse = mean_squared_error(y_test_tensor, test_outputs)
    r2 = r2_score(y_test_tensor, test_outputs)
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R2) Score: {r2:.4f}")
    
#error with tensor values --> loss values = NaN


In [40]:
df = pd.read_csv('/home/maatrika/Desktop/NasaL2/L1Data/Merged_Kp.csv') 
df = df.dropna()

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv('/home/maatrika/Desktop/NasaL2/L1Data/Merged_Kp.csv')  


df['Bx'] = pd.to_numeric(df['Bx'], errors='coerce')
df['By'] = pd.to_numeric(df['By'], errors='coerce')
df['Bz'] = pd.to_numeric(df['Bz'], errors='coerce')

numeric_columns = df.select_dtypes(include=[np.number])
X = numeric_columns.drop(columns=['Kp'])  
y = numeric_columns['Kp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

selected_features = ['x', 'y', 'ac', 'ab', 'w', 'z', 'aa', 'Bx', 'By', 'Bz']


X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

#convert data to PyTorch tensors and add an extra dimension for sequence length
X_train_tensor = torch.unsqueeze(torch.tensor(X_train_selected.values, dtype=torch.float32), 1)
X_test_tensor = torch.unsqueeze(torch.tensor(X_test_selected.values, dtype=torch.float32), 1)
y_train_tensor = torch.unsqueeze(torch.tensor(y_train.values, dtype=torch.float32), 1)
y_test_tensor = torch.unsqueeze(torch.tensor(y_test.values, dtype=torch.float32), 1)


class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        out, _ = self.gru(x)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out


input_size = len(selected_features)
hidden_size = 128
num_layers = 2
output_size = 1

model = GRUModel(input_size, hidden_size, num_layers, output_size)

#define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

#training loop
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    #backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():
    test_outputs = model(X_test_tensor)
    
    if torch.isnan(y_test_tensor).any() or torch.isnan(test_outputs).any():
        print("Warning: NaN values detected in the tensors.")
    
    mse = mean_squared_error(y_test_tensor.numpy(), test_outputs.numpy())
    r2 = r2_score(y_test_tensor.numpy(), test_outputs.numpy())
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R2) Score: {r2:.4f}")
    
    
#erroe being input in 2-D but tensors as 3-D


In [35]:
print(df.columns)


Index(['Time', 'Bx', 'By', 'Bz', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w',
       'x', 'y', 'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai',
       'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au',
       'av', 'aw', 'ax', 'Kp'],
      dtype='object')
